Este notebook demonstra uma solução para a competição de Preços de Habitação usando XGBoost e técnicas de pré-processamento padrão.



In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

## Carregando os dados

In [2]:
# Load the data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Separate target from predictors
y = train_data.SalePrice
X = train_data.drop(['SalePrice'], axis=1)

# Drop columns with too many missing values or irrelevant (Id)
X.drop(['Id'], axis=1, inplace=True)
test_ids = test_data['Id']
test_data.drop(['Id'], axis=1, inplace=True)

# Select numeric columns
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]

# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X.columns if X[cname].nunique() < 10 and 
                        X[cname].dtype == "object"]

# Keep selected columns only
my_cols = numerical_cols + categorical_cols
X = X[my_cols]
X_test = test_data[my_cols]

## Preprocessamento e modelagem dos dados

In [3]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)])

## Treinamento e validação

In [4]:
# Split data into train and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))

MAE: 17514.986232876712


## Fazendo a predição.

In [5]:
# Preprocessing of test data, fit model
preds_test = clf.predict(X_test)

# Save test predictions to file
output = pd.DataFrame({'Id': test_ids,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)
print("Submission saved to submission.csv")

Submission saved to submission.csv
